<a href="https://colab.research.google.com/github/Devasy23/FaceRec/blob/feature-triplet-loss-function/Model-Training/R50_finetune_m10" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

print(tf.__version__)

2.15.0


In [3]:
!pip install deepface
import deepface

In [4]:
# import the necessary packages
from tensorflow.keras.applications import resnet
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
def get_embedding_module(imageSize):
    # construct the input layer and pass the inputs through a
    # pre-processing layer
    inputs = keras.Input(imageSize + (3,))
    x = resnet.preprocess_input(inputs)

    # fetch the pre-trained resnet 50 model and freeze the weights
    baseCnn = resnet.ResNet50(weights="imagenet", include_top=False)
    baseCnn.trainable=False

    # pass the pre-processed inputs through the base cnn and get the
    # extracted features from the inputs
    extractedFeatures = baseCnn(x)
    # pass the extracted features through a number of trainable layers
    x = layers.GlobalAveragePooling2D()(extractedFeatures)
    x = layers.Dense(units=1024, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(units=128)(x)
    # build the embedding model and return it
    embedding = keras.Model(inputs, outputs, name="embedding")
    return embedding

In [5]:
def get_siamese_network(imageSize, embeddingModel):
    # build the anchor, positive and negative input layer
    anchorInput = keras.Input(name="anchor", shape=imageSize + (3,))
    positiveInput = keras.Input(name="positive", shape=imageSize + (3,))
    negativeInput = keras.Input(name="negative", shape=imageSize + (3,))
    # embed the anchor, positive and negative images
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    # build the siamese network and return it
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    return siamese_network

In [6]:
class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker
    def _compute_distance(self, inputs):
        (anchor, positive, negative) = inputs
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )

        # return the distances
        return (apDistance, anDistance)
    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss
    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        (apDistance, anDistance) = self._compute_distance(inputs)
        return (apDistance, anDistance)
    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            (apDistance, anDistance) = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        (apDistance, anDistance) = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)

        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    @property
    def metrics(self):
        return [self.lossTracker]

## Failed Try1

In [ ]:
# before training the model

# initialize the image size and the batch size
imageSize = (224, 224)

# initialize the margin and loss tracker
margin = 0.6
lossTracker = tf.keras.metrics.Mean(name="loss")

embedding_model = get_embedding_module(imageSize)
siamese_network = get_siamese_network(imageSize, embedding_model)
siamese_network.build((None, *imageSize, 3))
siamese_model = SiameseModel(siamese_network, margin, lossTracker)

# compile the model
siamese_model.compile(optimizer=keras.optimizers.Adam(1e-4))
siamese_network.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 anchor (InputLayer)         [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 positive (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 negative (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 embedding (Functional)      (None, 128)                  2638105   ['anchor[0][0]',              
                                                          6          'positive[0][0]',      

In [7]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def triplet_generator(base_dir="data", image_size=(224, 224)):
    """
    A generator that yields triplets: anchor, positive, and negative images.

    Parameters:
    - base_dir: The base directory where the images are stored.
    - image_size: The target size to which the images will be resized.

    Yields:
    - A tuple of numpy arrays: (anchor, positive, negative).
    """
    # Paths to the class directories
    ben_dir = os.path.join(base_dir, "ben")
    henry_dir = os.path.join(base_dir, "henry")

    # List of image filenames for each class
    ben_images = [os.path.join(ben_dir, img) for img in os.listdir(ben_dir)]
    henry_images = [os.path.join(henry_dir, img) for img in os.listdir(henry_dir)]

    while True:
        # Randomly choose an anchor and a positive image from the same class
        anchor_positive_class = np.random.choice(["ben", "henry"])
        if anchor_positive_class == "ben":
            anchor_path = np.random.choice(ben_images)
            positive_path = np.random.choice([img for img in ben_images if img != anchor_path])
            negative_path = np.random.choice(henry_images)
        else:
            anchor_path = np.random.choice(henry_images)
            positive_path = np.random.choice([img for img in henry_images if img != anchor_path])
            negative_path = np.random.choice(ben_images)

        # Load and preprocess the images
        anchor_img = img_to_array(load_img(anchor_path, target_size=image_size))
        positive_img = img_to_array(load_img(positive_path, target_size=image_size))
        negative_img = img_to_array(load_img(negative_path, target_size=image_size))

        # Normalize the images
        anchor_img /= 255.0
        positive_img /= 255.0
        negative_img /= 255.0

        yield ((anchor_img, positive_img, negative_img), None)

In [8]:
# now code to train the model
import numpy as np

# initialize the image size and the batch size
imageSize = (224, 224)
batchSize = 32

TRAIN_DIR = "data"
# initialize the training data generator
trainGen = triplet_generator(TRAIN_DIR, image_size=imageSize)


# Calculate steps per epoch assuming you know the total number of images
# For simplicity, let's assume each class in TRAIN_DIR has 3 images, making 6 images total.
# Adjust this calculation based on your actual dataset size.
num_classes = 2  # Adjust based on your dataset
images_per_class = 3  # Adjust based on your dataset
total_images = num_classes * images_per_class
steps_per_epoch = max(1, total_images // batchSize)  # This is a simplified calculation

# Train the model
# Note: You might need to adjust steps_per_epoch based on your actual data and how your generator works
history = siamese_model.fit(trainGen, epochs=10, steps_per_epoch=steps_per_epoch)


NameError: name 'siamese_model' is not defined

## Try2

In [10]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean

class TripletDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size, num_classes):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(labels)
        self.image_data_generator = ImageDataGenerator(preprocessing_function=resnet.preprocess_input)
        self.on_epoch_end()
        print(f"Initialized TripletDataGenerator with {len(self.image_paths)} images")

    def __len__(self):
        return max(1, len(self.image_paths) // self.batch_size)  # Ensure at least one batch

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.encoded_labels[index * self.batch_size:(index + 1) * self.batch_size]
        return self._generate_triplet_batch(batch_image_paths, batch_labels)

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        combined = list(zip(self.image_paths, self.encoded_labels))
        np.random.shuffle(combined)
        self.image_paths[:], self.encoded_labels[:] = zip(*combined)

    def _generate_triplet_batch(self, batch_image_paths, batch_labels):
        anchor_images = []
        positive_images = []
        negative_images = []

        for i in range(len(batch_image_paths)):
            anchor_path = batch_image_paths[i]
            anchor_label = batch_labels[i]

            positive_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l == anchor_label]
            )
            negative_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l != anchor_label]
            )

            anchor_image = load_img(anchor_path, target_size=self.image_size)
            positive_image = load_img(positive_path, target_size=self.image_size)
            negative_image = load_img(negative_path, target_size=self.image_size)

            anchor_images.append(img_to_array(anchor_image))
            positive_images.append(img_to_array(positive_image))
            negative_images.append(img_to_array(negative_image))

        return (
            (
                np.array(anchor_images),
                np.array(positive_images),
                np.array(negative_images)
            ),
            None,
        )

# Set the directory structure
data_dir = 'lfw'
image_size = (224, 224)
batch_size = 2  # Adjust the batch size for the small dataset
margin = 1.0

# Load and preprocess the data
image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)

# Debugging output
print(f"Found {len(image_paths)} images in {len(set(labels))} classes")

# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Check if the splits are non-empty
print(f"Training on {len(train_paths)} images")
print(f"Validating on {len(val_paths)} images")

# Create data generators
num_classes = len(set(labels))
train_generator = TripletDataGenerator(train_paths, train_labels, batch_size, image_size, num_classes)
val_generator = TripletDataGenerator(val_paths, val_labels, batch_size, image_size, num_classes)

# Check if the generators have data
assert len(train_generator) > 0, "Training generator is empty!"
assert len(val_generator) > 0, "Validation generator is empty!"

# Create the embedding model and the Siamese network
embedding_model = get_embedding_module(image_size)
siamese_network = get_siamese_network(image_size, embedding_model)

# Initialize the Siamese model
loss_tracker = Mean(name="loss")
siamese_model = SiameseModel(siamese_network, margin, loss_tracker)

# Compile the model
siamese_model.compile(optimizer=Adam())

# Train the model
siamese_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)


Found 6733 images in 610 classes
Training on 5386 images
Validating on 1347 images
Initialized TripletDataGenerator with 5386 images
Initialized TripletDataGenerator with 1347 images
Epoch 1/20


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "<ipython-input-6-85c7613062d8>", line 37, in train_step
        (apDistance, anDistance) = self._compute_distance(inputs)
    File "<ipython-input-6-85c7613062d8>", line 8, in _compute_distance
        (anchor, positive, negative) = inputs

    ValueError: not enough values to unpack (expected 3, got 1)


In [8]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xzvf lfw.tgz
import os
import shutil
count=0
for folder in os.listdir('lfw'):
    if len(os.listdir(f'lfw/{folder}')) < 4:
        shutil.rmtree(f'lfw/{folder}')
        count+=1
print(f"Removed {count} folders")

Streaming output truncated to the last 5000 lines.
lfw/Luis_Sanchez/Luis_Sanchez_0001.jpg
lfw/Juliette_Binoche/
lfw/Juliette_Binoche/Juliette_Binoche_0001.jpg
lfw/Joel_Todd/
lfw/Joel_Todd/Joel_Todd_0001.jpg
lfw/Allan_Wagner/
lfw/Allan_Wagner/Allan_Wagner_0001.jpg
lfw/Alejandro_Fernandez/
lfw/Alejandro_Fernandez/Alejandro_Fernandez_0001.jpg
lfw/Guillaume_Depardieu/
lfw/Guillaume_Depardieu/Guillaume_Depardieu_0001.jpg
lfw/Delphine_Chuillot/
lfw/Delphine_Chuillot/Delphine_Chuillot_0001.jpg
lfw/Paul_LeClerc/
lfw/Paul_LeClerc/Paul_LeClerc_0001.jpg
lfw/Eddie_Jordan/
lfw/Eddie_Jordan/Eddie_Jordan_0001.jpg
lfw/Lionel_Richie/
lfw/Lionel_Richie/Lionel_Richie_0001.jpg
lfw/Lionel_Richie/Lionel_Richie_0002.jpg
lfw/Jose_Viegas_Filho/
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0001.jpg
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0002.jpg
lfw/Roger_Mahony/
lfw/Roger_Mahony/Roger_Mahony_0001.jpg
lfw/Clay_Aiken/
lfw/Clay_Aiken/Clay_Aiken_0001.jpg
lfw/Clay_Aiken/Clay_Aiken_0002.jpg
lfw/Clay_Aiken/Clay_Aiken_000

## Fling 1

In [11]:
class TripletDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size, num_classes):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(labels)
        self.image_data_generator = ImageDataGenerator(preprocessing_function=resnet.preprocess_input)
        self.on_epoch_end()
        print(f"Initialized TripletDataGenerator with {len(self.image_paths)} images")

    def __len__(self):
        return max(1, len(self.image_paths) // self.batch_size)  # Ensure at least one batch

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.encoded_labels[index * self.batch_size:(index + 1) * self.batch_size]
        return self._generate_triplet_batch(batch_image_paths, batch_labels)

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        combined = list(zip(self.image_paths, self.encoded_labels))
        np.random.shuffle(combined)
        self.image_paths[:], self.encoded_labels[:] = zip(*combined)

    def _generate_triplet_batch(self, batch_image_paths, batch_labels):
        anchor_images = []
        positive_images = []
        negative_images = []

        for i in range(len(batch_image_paths)):
            anchor_path = batch_image_paths[i]
            anchor_label = batch_labels[i]

            positive_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l == anchor_label]
            )
            negative_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l != anchor_label]
            )

            anchor_image = load_img(anchor_path, target_size=self.image_size)
            positive_image = load_img(positive_path, target_size=self.image_size)
            negative_image = load_img(negative_path, target_size=self.image_size)

            anchor_images.append(img_to_array(anchor_image))
            positive_images.append(img_to_array(positive_image))
            negative_images.append(img_to_array(negative_image))

        return (
            {
                "anchor": np.array(anchor_images),
                "positive": np.array(positive_images),
                "negative": np.array(negative_images)
            },
            None,
        )


In [12]:
class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker

    def _compute_distance(self, inputs):
        anchor, positive, negative = inputs["anchor"], inputs["positive"], inputs["negative"]
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )
        # return the distances
        return (apDistance, anDistance)

    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        return (apDistance, anDistance)

    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            apDistance, anDistance = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}

    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}

    @property
    def metrics(self):
        return [self.lossTracker]

    def get_config(self):
        config = super().get_config()
        config.update({
            "siameseNetwork": self.siameseNetwork,
            "margin": self.margin,
            "lossTracker": self.lossTracker
            })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [15]:
# Set the directory structure
data_dir = 'lfw'
image_size = (224, 224)
batch_size = 2  # Adjust the batch size for the small dataset
margin = 10.0
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

# Initialize W&B
wandb.init(project="FaceRec", config={
    "learning_rate": 0.001,
    "epochs": 20,
    "batch_size": 2,
    "optimizer": "Adam",
    "architecture": "ResNet50",
    "dataset": "lfw",
    "loss": "TripletLoss",
    "margin": 10.0
})
# Load and preprocess the data
image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)

# Debugging output
print(f"Found {len(image_paths)} images in {len(set(labels))} classes")

# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Check if the splits are non-empty
print(f"Training on {len(train_paths)} images")
print(f"Validating on {len(val_paths)} images")

# Create data generators
num_classes = len(set(labels))
train_generator = TripletDataGenerator(train_paths, train_labels, batch_size, image_size, num_classes)
val_generator = TripletDataGenerator(val_paths, val_labels, batch_size, image_size, num_classes)

# Check if the generators have data
assert len(train_generator) > 0, "Training generator is empty!"
assert len(val_generator) > 0, "Validation generator is empty!"

# Create the embedding model and the Siamese network
embedding_model = get_embedding_module(image_size)
siamese_network = get_siamese_network(image_size, embedding_model)

# Initialize the Siamese model
loss_tracker = Mean(name="loss")
siamese_model = SiameseModel(siamese_network, margin, loss_tracker)

# Compile the model
siamese_model.compile(optimizer=Adam())

# Train the model
siamese_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[WandbMetricsLogger(log_freq=5)] # Remove options and add save_weights_only
)


Found 6733 images in 610 classes
Training on 5386 images
Validating on 1347 images
Initialized TripletDataGenerator with 5386 images
Initialized TripletDataGenerator with 1347 images
Epoch 1/20
2693/2693 [==============================] - 154s 52ms/step - loss: 7.0561 - val_loss: 4.4046
Epoch 2/20
2693/2693 [==============================] - 138s 51ms/step - loss: 5.9034 - val_loss: 4.2704
Epoch 3/20
2693/2693 [==============================] - 137s 51ms/step - loss: 4.8754 - val_loss: 3.4669
Epoch 4/20
2693/2693 [==============================] - 138s 51ms/step - loss: 4.6209 - val_loss: 3.2998
Epoch 5/20
2693/2693 [==============================] - 153s 57ms/step - loss: 4.1323 - val_loss: 3.5087
Epoch 6/20
2693/2693 [==============================] - 136s 51ms/step - loss: 3.6605 - val_loss: 2.4254
Epoch 7/20
2693/2693 [==============================] - 137s 51ms/step - loss: 3.3921 - val_loss: 2.2802
Epoch 8/20
2693/2693 [==============================] - 136s 51ms/step - loss: 3.0

In [18]:
# embedding_model.save('R50_m10_e20.h5')
siamese_model.save_weights('SM_R50_m10_e20.h5')

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import wandb
from wandb.keras import WandbMetricsLogger

class FineTuneCallback(keras.callbacks.Callback):
    def __init__(self, base_model, patience=5, unfreeze_layers=10):
        super(FineTuneCallback, self).__init__()
        self.base_model = base_model
        self.patience = patience
        self.unfreeze_layers = unfreeze_layers
        self.best_weights = None
        self.best_loss = float('inf')
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('val_loss')
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.best_weights = self.model.get_weights()
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                # Restore the best weights
                self.model.set_weights(self.best_weights)
                self.wait = 0
                # Unfreeze the last few layers
                for layer in self.base_model.layers[-self.unfreeze_layers:]:
                    if hasattr(layer, 'trainable'):
                        layer.trainable = True
                # Recompile the model to apply the changes
                self.model.compile(optimizer=Adam(learning_rate=1e-5))

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint("best_embedding_model.h5", save_best_only=True, save_weights_only=True, monitor='val_loss', verbose=1)
fine_tune_callback = FineTuneCallback(embedding_model, patience=5, unfreeze_layers=10)

for layer in embedding_model.layers:
    layer.trainable = False
# unfreeze last 20 layers
for layer in embedding_model.layers[-20:]:
    layer.trainable = True

# Compile the model
image_size = (224, 224)
embedding_model = get_embedding_module(image_size)
siamese_network = get_siamese_network(image_size, embedding_model)

# Initialize the Siamese model
loss_tracker = Mean(name="loss")
siamese_model = SiameseModel(siamese_network, margin, loss_tracker)

# Compile the model
siamese_model.compile(optimizer=Adam())
siamese_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=40,initial_epoch=20,
    callbacks=[reduce_lr, early_stopping, model_checkpoint, fine_tune_callback, WandbMetricsLogger(log_freq=5)]
)

Epoch 21/40
2692/2693 [============================>.] - ETA: 0s - loss: 7.1523
Epoch 21: val_loss improved from inf to 4.06779, saving model to best_embedding_model.h5
2693/2693 [==============================] - 153s 53ms/step - loss: 7.1519 - val_loss: 4.0678 - lr: 0.0010
Epoch 22/40
2693/2693 [==============================] - ETA: 0s - loss: 5.8743
Epoch 22: val_loss improved from 4.06779 to 3.95978, saving model to best_embedding_model.h5
2693/2693 [==============================] - 138s 51ms/step - loss: 5.8743 - val_loss: 3.9598 - lr: 0.0010
Epoch 23/40
2693/2693 [==============================] - ETA: 0s - loss: 5.3279
Epoch 23: val_loss improved from 3.95978 to 3.54885, saving model to best_embedding_model.h5
2693/2693 [==============================] - 137s 51ms/step - loss: 5.3279 - val_loss: 3.5489 - lr: 0.0010
Epoch 24/40
2693/2693 [==============================] - ETA: 0s - loss: 4.5554
Epoch 24: val_loss improved from 3.54885 to 3.18271, saving model to best_embedding_

In [20]:
!pip install tensorflow==2.15.0

In [14]:
!pip install wandb
import wandb
wandb.login()

wandb: Currently logged in as: 20bce057 (devasy). Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import resnet
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
from sklearn.preprocessing import LabelEncoder

# Initialize W&B
wandb.init(project="FaceRec", config={
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 2,
    "optimizer": "Adam",
    "architecture": "ResNet50",
    "dataset": "lfw",
    "loss": "TripletLoss",
    "margin": 0.6
})
config = wandb.config
def get_embedding_module(imageSize):
    # construct the input layer and pass the inputs through a
    # pre-processing layer
    inputs = keras.Input(imageSize + (3,))
    x = resnet.preprocess_input(inputs)

    # fetch the pre-trained resnet 50 model and freeze the weights
    baseCnn = resnet.ResNet50(weights="imagenet", include_top=False)
    baseCnn.trainable = False

    # pass the pre-processed inputs through the base cnn and get the
    # extracted features from the inputs
    extractedFeatures = baseCnn(x)
    # pass the extracted features through a number of trainable layers
    x = layers.GlobalAveragePooling2D()(extractedFeatures)
    x = layers.Dense(units=1024, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(units=128)(x)
    # build the embedding model and return it
    embedding = keras.Model(inputs, outputs, name="embedding")
    return embedding

def get_siamese_network(imageSize, embeddingModel):
    # build the anchor, positive and negative input layer
    anchorInput = keras.Input(name="anchor", shape=imageSize + (3,))
    positiveInput = keras.Input(name="positive", shape=imageSize + (3,))
    negativeInput = keras.Input(name="negative", shape=imageSize + (3,))
    # embed the anchor, positive and negative images
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    # build the siamese network and return it
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    return siamese_network

class TripletDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, image_size, num_classes):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(labels)
        self.image_data_generator = ImageDataGenerator(preprocessing_function=resnet.preprocess_input)
        self.on_epoch_end()
        print(f"Initialized TripletDataGenerator with {len(self.image_paths)} images")

    def __len__(self):
        return max(1, len(self.image_paths) // self.batch_size)  # Ensure at least one batch

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.encoded_labels[index * self.batch_size:(index + 1) * self.batch_size]
        return self._generate_triplet_batch(batch_image_paths, batch_labels)

    def on_epoch_end(self):
        # Shuffle the data at the end of each epoch
        combined = list(zip(self.image_paths, self.encoded_labels))
        np.random.shuffle(combined)
        self.image_paths[:], self.encoded_labels[:] = zip(*combined)

    def _generate_triplet_batch(self, batch_image_paths, batch_labels):
        anchor_images = []
        positive_images = []
        negative_images = []

        for i in range(len(batch_image_paths)):
            anchor_path = batch_image_paths[i]
            anchor_label = batch_labels[i]

            positive_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l == anchor_label]
            )
            negative_path = np.random.choice(
                [p for p, l in zip(self.image_paths, self.encoded_labels) if l != anchor_label]
            )

            anchor_image = load_img(anchor_path, target_size=self.image_size)
            positive_image = load_img(positive_path, target_size=self.image_size)
            negative_image = load_img(negative_path, target_size=self.image_size)

            anchor_images.append(img_to_array(anchor_image))
            positive_images.append(img_to_array(positive_image))
            negative_images.append(img_to_array(negative_image))

        return (
            {
                "anchor": np.array(anchor_images),
                "positive": np.array(positive_images),
                "negative": np.array(negative_images)
            },
            None,
        )

class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker

    def _compute_distance(self, inputs):
        anchor, positive, negative = inputs["anchor"], inputs["positive"], inputs["negative"]
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )
        # return the distances
        return (apDistance, anDistance)

    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        return (apDistance, anDistance)

    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            apDistance, anDistance = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        wandb.log({"loss": self.lossTracker.result()})  # Log loss to W&B
        return {"loss": self.lossTracker.result()}

    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        apDistance, anDistance = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        wandb.log({"val_loss": self.lossTracker.result()})  # Log validation loss to W&B
        return {"loss": self.lossTracker.result()}

    @property
    def metrics(self):
        return [self.lossTracker]

# Set the directory structure
data_dir = 'data'
image_size = (224, 224)
batch_size = 2  # Adjust the batch size for the small dataset
margin = 1.0

# Load and preprocess the data
image_paths = []
labels = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)

# Debugging output
print(f"Found {len(image_paths)} images in {len(set(labels))} classes")

# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Check if the splits are non-empty
print(f"Training on {len(train_paths)} images")
print(f"Validating on {len(val_paths)} images")

# Create data generators
num_classes = len(set(labels))
train_generator = TripletDataGenerator(train_paths, train_labels, batch_size, image_size, num_classes)
val_generator = TripletDataGenerator(val_paths, val_labels, batch_size, image_size, num_classes)

# Check if the generators have data
assert len(train_generator) > 0, "Training generator is empty!"
assert len(val_generator) > 0, "Validation generator is empty!"

# Create the embedding model and the Siamese network
embedding_model = get_embedding_module(image_size)
siamese_network = get_siamese_network(image_size, embedding_model)

# Initialize the Siamese model
loss_tracker = Mean(name="loss")
siamese_model = SiameseModel(siamese_network, margin, loss_tracker)

# Compile the model
siamese_model.compile(optimizer=Adam())

# Train the model with W&B callback
siamese_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[WandbMetricsLogger(log_freq=5), WandbModelCheckpoint("models")]
)

# Finish the W&B run
wandb.finish()


Found 6 images in 2 classes
Training on 4 images
Validating on 2 images
Initialized TripletDataGenerator with 4 images
Initialized TripletDataGenerator with 2 images
Epoch 1/20


ValueError: in user code:

    File "c:\Users\Devasy\OneDrive\Documents\GitHub\FaceRec\venv\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Devasy\OneDrive\Documents\GitHub\FaceRec\venv\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Devasy\OneDrive\Documents\GitHub\FaceRec\venv\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Devasy\AppData\Local\Temp\ipykernel_19076\2185040975.py", line 183, in train_step
        wandb.log({"loss": self.lossTracker.result()})  # Log loss to W&B
    File "c:\Users\Devasy\OneDrive\Documents\GitHub\FaceRec\venv\Lib\site-packages\wandb\sdk\wandb_run.py", line 449, in wrapper
        return func(self, *args, **kwargs)
    File "c:\Users\Devasy\OneDrive\